In [ ]:
#THIS IS FUCKED IN JUPYTER AND BREAKS SUPER
#%reload_ext autoreload
#%autoreload 2

import os, sys, pathlib

RunningInCOLAB = 'google.colab' in str(get_ipython())

In [ ]:
try:
    import torch
    print(torch.__version__)
    if torch.cuda.is_available():
        print(torch.cuda.get_device_properties(torch.cuda.get_device_name))
except UserWarning:
    pass

In [ ]:
if RunningInCOLAB:
    # move to content directory and remove directory for a clean start 
    %cd /content/         
    %rm -rf DataAugmentation
    # clone package repository (will throw error if already cloned)
    !git clone --branch new https://github.com/negril/Data-Augmentation.git DataAugmentation

In [ ]:
if RunningInCOLAB:
    # navigate to dl-for-har directory
    %cd /content/DataAugmentation/

# this statement is needed so that we can use the methods of the DL-ARC pipeline
module_path = str(pathlib.Path().resolve())
if module_path not in sys.path:
    sys.path.insert(1,module_path)

In [ ]:
if RunningInCOLAB:
    !pip install -r Requirements.txt -q
    !pip uninstall torchvision torchaudio -y
    !pip install --upgrade torch --extra-index-url https://download.pytorch.org/whl/cu113
else:
    !pip install -r Requirements.txt -q
    #--extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
import helper

data_complete = True

if os.path.isdir("./data"):
    for dir in helper.DEFAULT_RWHAR_FILEPATH:
        if not os.path.isdir(dir):
            data_complete = False
    if os.path.isdir("./data/PAMAP2_Dataset"):
        for dir in helper.DEFAULT_PAMAP2_FILEPATHS:
            if not os.path.isfile(dir):
                data_complete = False
else:
    data_complete = False

if not data_complete:
    %cd /content/DataAugmentation
    %rm -Rf /content/DataAugmentation/data
    !pip install --upgrade --no-cache-dir gdown
    import gdown
    id = '1XmM5Cm_Ow1WJytVDD364-__7dFYn002b'
    output = "data.tar"
    gdown.download(id=id, output=output, quiet=False)
    !tar --no-same-owner -xvf data.tar
    !rm data.tar
    
    %cd /content/DataAugmentation/data
    %mkdir PAMAP2_Dataset
    %mv PAMAP2 PAMAP2_Dataset/Protocol

    %cd /content/DataAugmentation

In [ ]:
import helper
if RunningInCOLAB:
    helper.WORKERS=4
else:
    helper.WORKERS=16

In [ ]:
import torch

device_properties=torch.cuda.get_device_properties(torch.cuda.get_device_name)
gencode=f"sm_{device_properties.major}{device_properties.minor}"

if gencode in torch.cuda.get_arch_list():
    print(f"Running on {torch.cuda.get_device_name()} with {gencode}")
    helper.ACCELERATOR="auto"
    helper.DEVICES="auto"
    print(f"{torch.cuda.get_device_properties(torch.cuda.get_device_name)}")
else:
    print(f"Can't run on {torch.cuda.get_device_name()} due to unsupported {gencode}, while torch supports: {torch.cuda.get_arch_list()}.")
    helper.ACCELERATOR="cpu"
    helper.DEVICES="auto"
    os.environ["CUDA_VISIBLE_DEVICES"]=""
    #torch.set_num_threads(6)

#helper.ACCELERATOR="cpu"
#helper.DEVICES="auto"
#os.environ["CUDA_VISIBLE_DEVICES"]=" "
#torch.set_num_threads(8)
#torch.set_num_interop_threads(16)
#torch.set_num_threads(4)
#torch.set_num_interop_threads(4)
#helper.WORKERS=4

print(f"Running on {helper.ACCELERATOR}.")   
print(f'threads: {torch.get_num_threads()} {torch.get_num_interop_threads()}')

In [ ]:
helper.WORKERS=16

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="max_split_size_mb:128"

In [ ]:
torch.cuda.empty_cache()

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(f"total {t/1024/1024}")
print(f"reser {r/1024/1024}")
print(f"alloc {a/1024/1024}")
print(f"freer {f/1024/1024}")

## DeepConvLSTM
### PAMAP2

In [ ]:
helper.train_LSTM_validation_model(
    helper.load_PAMAP2,
    total_activities = 7,
    val_pc = 0.3,
    batch_size = 32,
    data_size = (3, 9, 100),
    hidden_size = 256,
    lr = 0.0001,
    max_epochs = 30,
    tensorboard_save_dir = "Validation_LSTM_logs",
    tensorboard_name_prefix = "PAMAP2",
    monitor = "val_f1_score"
    )

### RWHAR

In [ ]:
helper.train_LSTM_validation_model(
    helper.load_RWHAR,
    total_activities = 8,
    val_pc = 0.3,
    batch_size = 128,
    data_size = (6, 50),
    hidden_size = 768,
    conv_filter = (5,5), 
    conv_padding = (2,2),
    lr = 0.0001,
    max_epochs = 100,
    tensorboard_save_dir = "Validation_LSTM_logs",
    tensorboard_name_prefix = "RWHAR",
    monitor = "val_f1_score"
    )

## Transformer
### PAMAP2

In [ ]:
helper.train_transformer_validation_model(
                helper.load_PAMAP2,
                total_activities = 7,
                val_pc = 0.3,
                batch_size = 32,
                data_size = (27, 100),
                nhead = 4,
                dim_feedforward = 2048,
                num_layer = 2,
                lr = 0.0001,
                max_epochs = 100,
                tensorboard_save_dir = "Validation_transformer_logs",
                tensorboard_name_prefix = "PAMAP2",
                monitor = "val_f1_score",)

### RWHAR

In [ ]:
helper.train_transformer_validation_model(
                helper.load_RWHAR,
                total_activities = 8,
                val_pc = 0.3,
                batch_size = 256,
                data_size = (6, 50),
                nhead = 5,
                dim_feedforward = 10000,
                dropout = 0.2,
                num_layer = 10,
                lr = 0.0001,
                max_epochs = 100,
                tensorboard_save_dir = "Validation_transformer_logs",
                tensorboard_name_prefix = "RWHAR",
                monitor = "val_f1_score",)

## Saved Model Score PAMAP2
Location : logs/PAMAP2_LSTM/version_2/checkpoints/epoch=8.ckpt

F1 Score : 96.21%

Loss     : 0.1168

## Parameters
LR           : 0.0001

Optimizer    : Adamax

Epochs       : 8

LR Scheduler : None

hidden_size : 256

## Saved Model Score
Location : epoch=56-val_loss=0.113-val_f1_score=0.970.ckpt

in_channels = 27, output_size = 7, d_model = 100, nhead = 4, num_layers = 2, dim_feedforward = 2048, 

## Saved model RWHAR

### Transformer Classifier

location : Validation_transformer_logs/RWHAR/version_3/checkpoints/epoch=63-val_loss=0.250-val_f1_score=0.926.ckpt

in_channels = 6, output_size = 8, d_model = 50, nhead = 5, dim_feedforward = 10000, num_layers = 8, dropout = 0.2

### PE model

epoch=78-val_loss=0.242-val_f1_score=0.928.ckpt

in_channels = 6, output_size = 8, d_model = 50, nhead = 5, dim_feedforward = 10000, num_layers = 10, dropout = 0.2

### LSTM model

location : Validation_LSTM_logs/RWHAR/version_0/checkpoints/epoch=25-val_loss=0.259-val_f1_score=0.918.ckpt

in_channels = 6,input_size = 50, hidden_size = 768, output_size = 8, conv_filter = (5,5), conv_padding = (2,2)


In [ ]:
#%load_ext autoreload
#%autoreload 2

import helper
import F1_score_check

## LSTM GAN Training

### PAMAP2

In [ ]:
from DeepConvLSTM_model import DeepConvNet
val_model = helper.load_pl_model("logs/Validation_LSTM_logs/PAMAP2/version_1/checkpoints/epoch=18-val_loss=0.121-val_f1_score=0.965.ckpt", DeepConvNet, hidden_size = 256)
helper.train_LSTM_GAN(helper.load_PAMAP2_activity,
                     val_model,
                     start_activity = 1,
                     total_activities = 7,
                     val_iter_size = 3, 
                     batch_size = 20,
                     data_size = (27, 100),
                     noise_len = 100,
                     gen_num_layers = 2,
                     gen_bidirectional = False,
                     dis_hidden_size = 100,
                     dis_num_layers = 2,
                     dis_bidirectional = False,
                     tensorboard_save_dir = "LSTM_GAN_logs",
                     tensorboard_name_prefix = "PAMAP2_act_",
                     )

### RWHAR

In [ ]:
from DeepConvLSTM_model import DeepConvNet
val_model = helper.load_pl_model("RWHAR_LSTM_918.ckpt", DeepConvNet, 
                                 in_channels = 6,input_size = 50, hidden_size = 768, 
                                 output_size = 8, conv_filter = (5,5), conv_padding = (2,2))
helper.train_LSTM_GAN(helper.load_RWHAR_activity,
                     val_model,
                     start_activity = 1,
                     total_activities = 8,
                     val_iter_size = 3, 
                     batch_size = 64,
                     data_size = (6, 50),
                     noise_len = 100,
                     gen_num_layers = 1,
                     gen_bidirectional = False,
                     dis_hidden_size = 200,
                     dis_num_layers = 2,
                     dis_bidirectional = False,
                     decay = 1,
                     tensorboard_save_dir = "LSTM_GAN_logs",
                     tensorboard_name_prefix = "RWHAR_act_",
                     )


## Transformer GAN
### PAMAP2

In [ ]:
from TransformerClassifier import TransformerClassifier
# val_model = helper.load_pl_model("val_model_transformer.ckpt", TransformerClassifier, 
#                                  in_channels = 27, output_size = 7, 
#                                  d_model = 100, nhead = 1, dim_feedforward = 2048,)
val_model = helper.load_pl_model("logs/Validation_transformer_logs/PAMAP2/version_2/checkpoints/epoch=42-val_loss=0.123-val_f1_score=0.970.ckpt", TransformerClassifier, 
                                 in_channels = 27, output_size = 7, 
                                 d_model = 100, nhead = 4, dim_feedforward = 2048, num_layers = 2,)
helper.train_transformer_GAN(
    helper.load_PAMAP2_activity,
    val_model,
    start_activity = 1,
    total_activities = 7,
    val_iter_size = 3,
    batch_size = 20,
    data_size = (27, 100),
    noise_len = 100,
    period = 100,
    max_retries = 2,
    init_dim_feedforward = 2048,
    dim_feedforward_exponent = 5,
    gen_nheads = 5,
    dis_nheads = 5,
    tensorboard_save_dir = "transformer_GAN_logs",
    tensorboard_name_prefix = "PAMAP2_act_",
    )

### RWHAR

In [ ]:
from TransformerClassifier import TransformerClassifier
# val_model = helper.load_pl_model("RWHAR_transformer_10layer.ckpt", TransformerClassifier, 
#                                  in_channels = 6, output_size = 8, 
#                                  d_model = 50, nhead = 5, dim_feedforward = 10000, num_layers = 10)
# from Validation_model import Net
# val_model = Net.load_from_checkpoint("RWHAR_transformer_fixed.ckpt") 
val_model = helper.load_pl_model("RWHAR_transformer_pe.ckpt", TransformerClassifier, 
                                 in_channels = 6, output_size = 8, 
                                 d_model = 50, nhead = 5, dim_feedforward = 10000, num_layers = 10)

helper.train_transformer_GAN(
    helper.load_RWHAR_activity,
    val_model,
    start_activity = 1,
    total_activities = 8,
    val_iter_size = 3,
    batch_size = 128,
    data_size = (6, 50),
    noise_len = 100,
    period = 100,
    max_retries = 2,
    init_dim_feedforward = 4000,
    dim_feedforward_exponent = 2,
    gen_nheads = 5,
    dis_heads = 5,
    gen_num_layers = 5,
    dis_num_layers = 5,
    tensorboard_save_dir = "transformer_GAN_logs",
    tensorboard_name_prefix = "RWHAR_act_",
    dis_lr = 0.0004,
    gen_lr = 0.0002,
    )

In [ ]:
## Things to Try to stabilise GAN
## 3. Use one sided label smoothing for discriminator (helps a little)
## 5. Remove Linear and use conv only (helps a ton)
## 6. Use dropout of 0.5 